## About Profile 👤

Profile Struct Defined

```
/**
* @notice A struct containing profile data.
*
* @param pubCount The number of publications made to this profile.
* @param followModule The address of the current follow module in use by this profile, can be empty.
* @param followNFT The address of the followNFT associated with this profile, can be empty..
* @param handle The profile's associated handle.
* @param imageURI The URI to be used for the profile's image.
* @param followNFTURI The URI to be used for the follow NFT.
*/
struct ProfileStruct {
    uint256 pubCount;
    address followModule;
    address followNFT;
    string handle;
    string imageURI;
    string followNFTURI;
}
```

Create Profile Struct

```
/**
* @notice A struct containing the parameters required for the `createProfile()` function.
*
* @param to The address receiving the profile.
* @param handle The handle to set for the profile, must be unique and non-empty.
* @param imageURI The URI to set for the profile image.
* @param followModule The follow module to use, can be the zero address.
* @param followModuleInitData The follow module initialization data, if any.
* @param followNFTURI The URI to use for the follow NFT.
*/
struct CreateProfileData {
    address to;
    string handle;
    string imageURI;
    address followModule;
    bytes followModuleInitData;
    string followNFTURI;
}
```

Function createProfile In `LensHub`

```
function createProfile(DataTypes.CreateProfileData calldata vars)
    external
    override
    whenNotPaused
    returns (uint256)
{
    if (!_profileCreatorWhitelisted[msg.sender]) revert Errors.ProfileCreatorNotWhitelisted();
    unchecked {
        uint256 profileId = ++_profileCounter;
        _mint(vars.to, profileId);
        PublishingLogic.createProfile(
            vars,
            profileId,
            _profileIdByHandleHash,
            _profileById,
            _followModuleWhitelisted
        );
        return profileId;
    }
}
```

Function createProfile In `PublishingLogic`

```
function createProfile(
        DataTypes.CreateProfileData calldata vars,
        uint256 profileId,
        mapping(bytes32 => uint256) storage _profileIdByHandleHash,
        mapping(uint256 => DataTypes.ProfileStruct) storage _profileById,
        mapping(address => bool) storage _followModuleWhitelisted
    ) external;
```

从上面的可以发现，`LensHub`是入口

首先需要判断铸造Profile的地址在白名单中
```
mapping(address => bool) internal _profileCreatorWhitelisted;
```

其中profileId的生成是以合约计数自增(++)的方式，向`to`地址`mint`了指定profileId的NFT

内部再通过其逻辑操作单元`PublishingLogic`中对应地方法进行profile的铸造,同时将具体的信息数据存储到`LensHubStorage`对应的变量中

关于`ProfileStruct`的`pubCount`的表达的是该`profileId`发布操作的计数索引